# TP1 - Pandas y Visualización de datos

N9. Queremos analizar qué tan fácil sería hacer un slogan para las películas en base a su descripción con machine learning. La primera pregunta consiste en entender: ¿Qué porcentaje de las palabras que no son stopwords de las que aparecen en los slogans aparecen también en la descripción? ¿Y qué porcentaje de las tuplas de dos palabras cumplen eso? ¿Y tuplas de tres? A mayores los porcentajes, más fácil será generar el slogan con la descripción.

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk import word_tokenize
nltk.download('wordnet')
nltk.download('punkt')
from nltk.util import ngrams
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
peliculas_original = pd.read_csv("/content/drive/MyDrive/Organización de Datos/TP1/Movie dataset/movies.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
pelis = peliculas_original[ ["title", "tagline", "overview"] ].dropna()

stopwords_en = set(stopwords.words('english'))
puntuacion = string.punctuation + '"' + "..." + "``"

def sacar_stopwords(slogan):
  slogan_tokens = word_tokenize(slogan)
  slogan_sin_stopwords = [t for t in slogan_tokens if t not in stopwords_en and t not in puntuacion]
  return slogan_sin_stopwords

pelis["tagline_sin_stopwords"] = pelis["tagline"].apply(sacar_stopwords)
pelis["overview_sin_stopwords"] = pelis["overview"].apply(sacar_stopwords)
pelis = pelis[pelis["overview_sin_stopwords"].str.len() > 0]

In [5]:
def porcentaje_comunes(tagline, overview):
  return (len(set(tagline).intersection(overview)) / len(overview)) * 100

def obtener_tuplas(tagline, overview, largo_tuplas):
  tuplas_tagline = list(ngrams(tagline, largo_tuplas))
  tuplas_overview = list(ngrams(overview, largo_tuplas))
  if len(tuplas_overview) == 0: tuplas_overview = [overview[0]] #Si no tiene tuplas, hay un ZeroDivisionError al calcular el porcentaje. Agrego una lista con un único elemento para dividir por 1.
  return porcentaje_comunes(tuplas_tagline, tuplas_overview)

pelis["porcentaje"] = pelis[["tagline_sin_stopwords", "overview_sin_stopwords"]].apply(lambda x: porcentaje_comunes(*x), axis=1)
pelis["porcentaje_tuplas2"] = pelis[["tagline_sin_stopwords", "overview_sin_stopwords"]].apply(lambda x: obtener_tuplas(*x, 2), axis=1)
pelis["porcentaje_tuplas3"] = pelis[["tagline_sin_stopwords", "overview_sin_stopwords"]].apply(lambda x: obtener_tuplas(*x, 3), axis=1)
pelis = pelis[ ["title", "tagline", "overview", "porcentaje", "porcentaje_tuplas2", "porcentaje_tuplas3"] ]
pelis

,title,tagline,overview,tagline_sin_stopwords,overview_sin_stopwords,porcentaje,porcentaje_tuplas2,porcentaje_tuplas3
1,Jumanji,Roll the dice and unleash the excitement!,When siblings Judy and Peter discover an encha...,"[Roll, dice, unleash, excitement]","[When, siblings, Judy, Peter, discover, enchan...",0.000000,0.000000,0.0
2,Grumpier Old Men,Still Yelling. Still Fighting. Still Ready for...,A family wedding reignites the ancient feud be...,"[Still, Yelling, Still, Fighting, Still, Ready...","[A, family, wedding, reignites, ancient, feud,...",0.000000,0.000000,0.0
3,Waiting to Exhale,Friends are the people who let you be yourself...,"Cheated on, mistreated and stepped on, the wom...","[Friends, people, let, never, let, forget]","[Cheated, mistreated, stepped, women, holding,...",3.703704,0.000000,0.0
4,Father of the Bride Part II,Just When His World Is Back To Normal... He's ...,Just when George Banks has recovered from his ...,"[Just, When, His, World, Is, Back, To, Normal,...","[Just, George, Banks, recovered, daughter, 's,...",10.000000,0.000000,0.0
5,Heat,A Los Angeles Crime Saga,"Obsessive master thief, Neil McCauley leads a ...","[A, Los, Angeles, Crime, Saga]","[Obsessive, master, thief, Neil, McCauley, lea...",5.555556,2.857143,0.0
...,...,...,...,...,...,...,...,...
45454,The Morning After,What happened last night?,The Morning After is a feature film that consi...,"[What, happened, last, night]","[The, Morning, After, feature, film, consists,...",0.000000,0.000000,0.0
45456,House of Horrors,Meet...The CREEPER!,An unsuccessful sculptor saves a madman named ...,"[Meet, The, CREEPER]","[An, unsuccessful, sculptor, saves, madman, na...",5.882353,0.000000,0.0
45458,The Burkittsville 7,"Do you know what happened 50 years before ""The...",A film archivist revisits the story of Rustin ...,"[Do, know, happened, 50, years, The, Blair, Wi...","[A, film, archivist, revisits, story, Rustin, ...",13.333333,7.142857,0.0
45461,Subdue,Rising and falling between a man and woman,Rising and falling between a man and woman.,"[Rising, falling, man, woman]","[Rising, falling, man, woman]",100.000000,100.000000,100.0
